In [1]:
from pulp import *
import pandas as pd

In [2]:
data = pd.read_csv(r'Data/dist_final.csv', sep=';', encoding = "ISO-8859-1")

In [118]:
# Se genera la matriz de costos de transporte
# i total nodos de suministro (70)
# j total nodos origen (1125)
ODMatrix = pd.DataFrame(columns = [list(range(1,71))])
ODMatrix.index += 1
j = 1
k = 0
d_node = []
for i in range(len(data)):  
    if data['ï»¿origin_id'][i] == j:
        d_node.append(data['total_cost'][i].replace(',','.'))        
    else:
        ODMatrix.loc[k] = d_node
        d_node = []
        j = j+1
        k = k+1
        d_node.append(data['total_cost'][i].replace(',','.'))
ODMatrix.loc[k] = d_node
ODMatrix.index += 1

In [119]:
#Se agrega la demanda como columna en la matriz OD
demanda = pd.read_csv(r'Data/centroides_f.csv', sep=';', encoding = "ISO-8859-1") 
ODMatrix['demanda'] = list(demanda['Poblacion'])
ODMatrix

,1,2,3,4,5,6,7,8,9,10,...,62,63,64,65,66,67,68,69,70,demanda
1,125.5390033,458.4985512,325.0361432,1569.066975,1631.734669,905.2065943,2846.450734,1468.371459,1366.739201,649.6738859,...,3578.184694,2680.283195,6257.757346,5252.458423,5501.172217,5452.687292,5210.036371,5409.179291,5435.803834,23
2,4158.395873,4426.522619,4293.060211,2809.585374,2872.253068,3383.394107,2394.216188,3090.593356,3262.703028,3672.29627,...,1930.999737,2228.048649,2266.963574,1054.069327,1302.78312,1442.381972,1570.309552,1770.87768,1910.498936,28
3,4529.522748,4797.649494,4664.187086,3180.71225,3243.379944,3754.520983,2493.58511,3461.720232,3633.829903,4043.423145,...,1705.695365,2591.838484,2182.120707,1106.276404,1354.990198,1494.589049,1622.51663,1823.084757,1962.706013,38
4,4950.941115,5219.067861,5085.605453,3602.130617,3664.798311,4175.93935,2874.148299,3883.138599,4055.24827,4464.841512,...,2086.258554,2972.401673,2120.206644,907.3123976,1156.026191,1295.625043,1423.552623,1624.12075,1763.742006,12
5,4508.588596,4776.715342,4643.252934,3159.778097,3222.445791,3733.58683,2431.79578,3440.786079,3612.895751,4022.488993,...,1643.906035,2530.049154,2120.331376,1160.958536,1409.672329,1549.271181,1677.198761,1877.766888,2017.388145,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,9114.547099,9382.673845,9249.211437,7765.736601,7828.404295,8339.545334,7023.665884,8046.744583,8218.854254,8628.447496,...,6235.776139,7121.919259,4958.213043,6193.606635,5921.340208,6360.786018,6663.087881,6562.266237,6565.230838,77
1122,4651.748903,4919.875649,4786.413241,3298.019508,3360.687202,3876.747137,1977.610025,3579.02749,3751.137162,4165.649299,...,1252.075356,2075.863399,4045.498973,3526.904127,3775.617921,3915.216772,3988.894139,4188.037059,4214.661601,77
1123,3600.507959,3868.634706,3735.172298,2251.697461,2314.365155,2825.506194,1836.328275,2532.705443,2704.815115,3114.408356,...,1607.609242,1670.160736,3460.394174,2445.576931,2694.290724,2645.8058,2403.154879,2602.297799,2628.922341,925
1124,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,...,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,2628.922341,7


In [300]:
#Se exporta la matriz OD como csv
#ODMatrix.to_csv('Data/ODMatrix.csv', sep = ";", encoding = "windows-1252")

In [98]:
#se cre la lista con los nodos de suministro y nodos de demanda
supply_n = list(range(1,71))
demand_n = list(range(1,1126))

In [120]:
# Se crea diccionario con los costos
Cij = {}
for i in range(len(demand_n)):
    Cij[demand_n[i]] = {}
    for j in range(1,71):
        Cij[demand_n[i]][demand_n[j-1]] = ODMatrix.iloc[i][j].item()

In [121]:
#Se crea diccionario con las demandas
Dj = {}
for k in range(len(demand_n)):
    Dj[demand_n[k]] = ODMatrix.iloc[k]['demanda'].item()
combinaciones = [(i,j) for i in demand_n for j in supply_n]

In [122]:
Xij = LpVariable.dicts('Se sirve',(demand_n,supply_n),0,1, LpBinary)
Yj = LpVariable.dicts('Se localiza en',supply_n,0,1,LpBinary)

In [116]:
float(Cij[1][1])

125.5390033

In [123]:
p_median = LpProblem("p_median", LpMinimize)
p_median += lpSum([float(Cij[i][j])*Xij[i][j] for (i,j) in combinaciones])

for j in supply_n:
    p_median += lpSum([Xij[i][j] for (i) in demand_n]) == 1 #Restriccion Sum(Xij) en i = 1; Para todo j

p_median += lpSum([Yj[j] for j in supply_n]) == 1

In [125]:
p_median.solve()
for v in p_median.variables():
    if v.varValue > 0:
        print(v.name,'=',v.varValue)

Se_localiza_en_63 = 1.0
Se_sirve_1019_53 = 1.0
Se_sirve_1041_28 = 1.0
Se_sirve_1042_32 = 1.0
Se_sirve_1042_33 = 1.0
Se_sirve_1042_62 = 1.0
Se_sirve_1045_41 = 1.0
Se_sirve_1054_13 = 1.0
Se_sirve_1056_1 = 1.0
Se_sirve_1062_21 = 1.0
Se_sirve_1063_30 = 1.0
Se_sirve_1068_60 = 1.0
Se_sirve_1071_42 = 1.0
Se_sirve_1077_5 = 1.0
Se_sirve_1079_31 = 1.0
Se_sirve_1088_10 = 1.0
Se_sirve_1123_52 = 1.0
Se_sirve_184_37 = 1.0
Se_sirve_213_20 = 1.0
Se_sirve_229_46 = 1.0
Se_sirve_255_25 = 1.0
Se_sirve_340_58 = 1.0
Se_sirve_348_22 = 1.0
Se_sirve_34_35 = 1.0
Se_sirve_367_38 = 1.0
Se_sirve_378_14 = 1.0
Se_sirve_392_39 = 1.0
Se_sirve_407_29 = 1.0
Se_sirve_411_65 = 1.0
Se_sirve_426_67 = 1.0
Se_sirve_443_9 = 1.0
Se_sirve_509_36 = 1.0
Se_sirve_514_64 = 1.0
Se_sirve_522_69 = 1.0
Se_sirve_527_23 = 1.0
Se_sirve_538_8 = 1.0
Se_sirve_552_40 = 1.0
Se_sirve_555_17 = 1.0
Se_sirve_642_12 = 1.0
Se_sirve_651_63 = 1.0
Se_sirve_651_7 = 1.0
Se_sirve_655_16 = 1.0
Se_sirve_655_18 = 1.0
Se_sirve_656_57 = 1.0
Se_sirve_673_68 = 1.